# **REQUIRED LIBRARIES**

In [ ]:
# !pip install stockstats
# !pip install pandas_ta
# !pip install plotly

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import math as mt
import warnings
import stockstats
warnings.filterwarnings('ignore')
import pandas_ta as ta
import plotly.express as px
import plotly.graph_objects as go

# Data Acquisition

In [3]:
data=pd.read_csv(r"data_2020_2023_1h.csv")

In [4]:
data["datetime"]=pd.to_datetime(data["datetime"])

In [ ]:
data

## stock closing price

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
# Plotting stock price
fig.add_trace(go.Scatter(x=data['datetime'], y=data['close'], mode='lines', name='Stock Price'))
# Adding layout details
fig.update_layout(
    title='Stock Closing Price Over Time',
    xaxis=dict(title='Date'),
    yaxis=dict(title='Stock Closing Price'),
    template='plotly_dark'
)
# Display the plot
fig.show()

# Data Visualisation

## Candelstick pattern of stock

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.7, 0.3])

candlestick = go.Candlestick(x=data['datetime'],
                             open=data['open'],
                             high=data['high'],
                             low=data['low'],
                             close=data['close'],
                             name='Candlesticks')

fig.add_trace(candlestick, row=1, col=1)

volume_bars = go.Bar(x=data['datetime'], y=data['volume'], name='Volume Bars', marker=dict(color='yellow'))

fig.add_trace(volume_bars, row=2, col=1)

fig.update_layout(title='Candlestick Chart with Volume Bars',
                  xaxis_title='Date',
                  yaxis_title='Stock Price',
                  template='plotly_dark')
fig.show()

## Average True Range (ATR):

The formula for Average True Range (ATR) is given by:
$$
\text{ATR}_t = \frac{1}{n} \sum_{i=0}^{n-1} \max\left( H_{t-i} - L_{t-i}, \, |H_{t-i} - C_{t-i-1}|, \, |L_{t-i} - C_{t-i-1}| \right)
$$
Where:

- \( H_t \) is the high price at time \( t \).
- \( L_t \) is the low price at time \( t \).
- \( C_{t-1} \) is the closing price at time \( t-1 \).


In [8]:
data['ATR'] = ta.atr(data['high'], data['low'], data['close'], length= 6)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(x=data.index, y=data['ATR'], name='ATR',
                     marker=dict(color='rgba(58, 71, 80, 0.6)',
                                 line=dict(color='rgba(255, 71, 80, 1.0)', width=2))))

fig.update_layout(title='ATR Bars',
                  xaxis_title='Date',
                  yaxis_title='ATR',
                  template='plotly_dark')

fig.show()

# Denoising Data

## Heiken Ashi Method

Heikin-Ashi Close:

$$
\text{HA_Close}_t = \frac{\text{Open}_t + \text{High}_t + \text{Low}_t + \text{Close}_t}{4}
$$

Heikin-Ashi Open:
$$
\text{HA_Open}_t = \frac{\text{HA_Open}_{t-1} + \text{HA_Close}_{t-1}}{2}
$$

Heikin-Ashi High:
$$
\text{HA_High}_t = \max(\text{High}_t, \text{HA_Open}_t, \text{HA_Close}_t)
$$

Heikin-Ashi Low:
$$
\text{HA_Low}_t = \min(\text{Low}_t, \text{HA_Open}_t, \text{HA\_Close}_t)
$$

In [10]:
def heikin_ashi_candlesticks(data: pd.DataFrame):
    # Calculate HA close
    data["HA_CLOSE"] = (data["low"] + data["high"] + data["close"] + data["open"]) / 4

    for i in range(len(data)):
            if i == 0:
                # For the first row
                data.at[i, "HA_OPEN"] = (data.at[i, "open"] + data.at[i, "close"]) / 2
            else:
                # For subsequent rows
                data.at[i, "HA_OPEN"] = (data.at[i - 1, "HA_OPEN"] + data.at[i - 1, "HA_CLOSE"]) / 2

    data["HA_HIGH"] = data[["HA_OPEN", "HA_CLOSE", "high"]].max(axis=1)
    data["HA_LOW"] = data[["HA_OPEN", "HA_CLOSE", "low"]].min(axis=1)

    return data
data=heikin_ashi_candlesticks(data)

In [ ]:
fig=px.line(data,x="datetime",y=["close","HA_CLOSE"],title="Denoising data",template="plotly_dark")
fig.show()

# STARTEGY

## MACD

* Moving average convergence/divergence (MACD, or MAC-D): It shows the relationship between two exponential moving averages (EMAs) of a security's price. The MACD line is calculated by subtracting the 28-period EMA from the 12-period EMA.

* Input Parameters- An array containing the closing price of the stock

In [12]:
def macd(data):

    data['MACD'] = data['HA_CLOSE'].ewm(span = 12,adjust=False).mean() - data['HA_CLOSE'].ewm(span=28,adjust=False).mean()
    data['Signal_MACD'] = data['MACD'].ewm(span = 9,adjust = False).mean()
    return data
data = macd(data)

In [ ]:
import plotly.graph_objects as go
import pandas as pd



data['MACD_Histogram'] = data['Signal_MACD'] - data['MACD']

trace_macd = go.Scatter(x=data.index, y=data['MACD'], mode='lines', name='MACD', line=dict(color='#4C78A8'))
trace_signal = go.Scatter(x=data.index, y=data['Signal_MACD'], mode='lines', name='Signal MACD', line=dict(color='#F58518'))
trace_histogram = go.Bar(x=data.index, y=data['MACD_Histogram'], name='MACD Histogram',
                         marker=dict(color=['red' if val < 0 else 'green' for val in data['MACD_Histogram']]))


layout = go.Layout(title='MACD and Signal Line Over Time',
                   xaxis=dict(title='Time'),
                   yaxis=dict(title='MACD'),
                   showlegend=True)

fig = go.Figure(data=[trace_macd, trace_signal, trace_histogram], layout=layout)
fig.show()

### Signal Generation based on MACD

* BUY SIGNAL(1): When MACD line crosses Signal Line from below
* SELL SIGNAL(-1): When MACD Line crosses  Signal Line from above

In [14]:
def signal_MACD(data):
    signals=[0]
    for i in range(1,data.shape[0]):
      if(data['MACD'][i] > data['Signal_MACD'][i] and data['MACD'][i-1] < data['Signal_MACD'][i-1]):
        signals.append(1)
      elif(data['MACD'][i] < data['Signal_MACD'][i] and data['MACD'][i-1] > data['Signal_MACD'][i-1]):
        signals.append(-1)
      else:
        signals.append(0)
    data["signals_MACD"]=signals
    return data
data=signal_MACD(data)

## ADX

* The average directional index (ADX) is a technical indicator used by traders to determine the strength of a price trend for a financial security. Trading in the direction of a strong trend reduces risk and increases profit potential. Many traders consider the ADX to be the ultimate trend indicator because it is so reliable.

* +DI (Positive Directional Indicator): Measures the presence of an upward trend. It indicates the strength of positive price movement.
* -DI (Negative Directional Indicator): Measures the presence of a downward trend. It indicates the strength of negative price movement.
* ADX (Average Directional Index): Smooths the DX (Directional Index) values to provide a single line that reflects the strength of the trend, without indicating its direction. An ADX value above 25 typically indicates a strong trend, while a value below 20 suggests a weak or non-existent trend.

In [15]:
def adx(data,period):
  ad = ta.adx(data['HA_HIGH'] , data['HA_LOW'] , data['HA_CLOSE'] , timeperiod=period)
  data['ADX'] = ad['ADX_14']
  data['+DI'] = ad['DMP_14']
  data['-DI'] = ad['DMN_14']
  return data
data=adx(data,14)

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=('ADX', '+DI and -DI'),
                    vertical_spacing=0.1)

# Add ADX trace
fig.add_trace(go.Scatter(x=data.index, y=data['ADX'], mode='lines', name='ADX',
                         line=dict(color='blue', width=2)), row=1, col=1)

# Add +DI and -DI traces
fig.add_trace(go.Scatter(x=data.index, y=data['+DI'], mode='lines', name='+DI',
                         line=dict(color='green', width=2)), row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['-DI'], mode='lines', name='-DI',
                         line=dict(color='red', width=2)), row=2, col=1)

# Update layout
fig.update_layout(title='Average Directional Index (ADX) with +DI and -DI',
                  template='plotly_dark')

# Update x-axis and y-axis labels
fig.update_xaxes(title_text='Date', row=2, col=1)
fig.update_yaxes(title_text='ADX Value', row=1, col=1)
fig.update_yaxes(title_text='Directional Indicators', row=2, col=1)

# Show plot
fig.show()

### signals generation using adx

In [17]:
def signal_adx(data):
  signals = []
  for i in range(len(data)):
    if(data['+DI'][i] > data['-DI'][i] and data['ADX'][i] >= 25):
      signals.append(1)
    elif(data['+DI'][i] < data['-DI'][i] and data['ADX'][i] >= 25):
      signals.append(-1)
    else:
      signals.append(0)
  data['signals_adx'] = signals
  return data
data=signal_adx(data)

## ICHIMOKU CLOUD

In [18]:
def ichimoku_cloud(
    data: pd.DataFrame,
    small_period: int = 7,
    large_period: int = 42,
    rolling_period: int = 60,
):

    small_period_high = data["HA_HIGH"].rolling(window=small_period).max()
    small_period_low = data["HA_LOW"].rolling(window=small_period).min()
    data["tenkan_sen"] = (small_period_high + small_period_low) / 2

    large_period_high = data["HA_HIGH"].rolling(window=large_period).max()
    large_period_low = data["HA_LOW"].rolling(window=large_period).min()
    data["kijun_sen"] = (large_period_high + large_period_low) / 2

    data["senkou_span_a"] = (data["tenkan_sen"] + data["kijun_sen"]) / 2
    data["senkou_span_a"] = data["senkou_span_a"].shift(20)
    data["senkou_span_b"] = (
        data["HA_HIGH"].rolling(window=rolling_period).max()
        + data["HA_LOW"].rolling(window=rolling_period).min()
    ) / 2
    data["senkou_span_b"] = data["senkou_span_b"].shift(40)
    cloud_upper_bound = []
    cloud_lower_bound = []
    cloud_width = []
    data['cloud_threshold'] = 0
    for i in range(len(data)):

        cloud_width.append(abs(data['senkou_span_a'][i] - data['senkou_span_b'][i]))
        cloud_upper_bound.append(max(data['senkou_span_a'][i],data['senkou_span_b'][i]))
        cloud_lower_bound.append(min(data['senkou_span_a'][i],data['senkou_span_b'][i]))
        data['cloud_threshold'][i] = (cloud_upper_bound[-1] - cloud_lower_bound[-1])
    cloud_threshold = data['cloud_threshold'].rolling(window=11).mean()
    return data
data = ichimoku_cloud(data)

In [ ]:
fig=px.line(data,x="datetime",y=["close","senkou_span_a","senkou_span_b"])
fig.show()

### SIGNAL GENERATION BY ICHIMOKU CLOUD

In [20]:
def signals_ichimoku(data):
  cloud_upper_bound = []
  cloud_lower_bound = []
  cloud_width = []
  data['cloud_threshold'] = 0
  for i in range(len(data)):

      cloud_width.append(abs(data['senkou_span_a'][i] - data['senkou_span_b'][i]))
      cloud_upper_bound.append(max(data['senkou_span_a'][i],data['senkou_span_b'][i]))
      cloud_lower_bound.append(min(data['senkou_span_a'][i],data['senkou_span_b'][i]))
      data['cloud_threshold'][i] = (cloud_upper_bound[-1] - cloud_lower_bound[-1])
  cloud_threshold = data['cloud_threshold'].rolling(window=11).mean()

  signals = []
  for i in range(len(data)):

      if(data['close'][i] > cloud_upper_bound[i]  and cloud_width[i] > 2*cloud_threshold[i]):
          signals.append(1)
      elif(data['close'][i] < cloud_lower_bound[i]  and cloud_width[i] > 2*cloud_threshold[i]):
          signals.append(-1)
      else:
          signals.append(0)
  data['signals_ich_1'] = signals

  ## Bearish and bullish market signalss

  signals = []
  for i in range(len(data)):
      if(data['tenkan_sen'][i] > data['kijun_sen'][i] and data['tenkan_sen'][i-1] < data['kijun_sen'][i-1]):
          signals.append(1)
      elif(data['tenkan_sen'][i] < data['kijun_sen'][i] and data['tenkan_sen'][i-1] > data['kijun_sen'][i-1]):
          signals.append(-1)
      else:
          signals.append(0)
  data['signals_ich_2'] = signals

  ## Another way of genrating signalss

  signals = []
  for i in range(len(data)):
      if(data.HA_CLOSE[i] > data.senkou_span_a[i] and data.senkou_span_a[i] > data.senkou_span_b[i]):
          signals.append(1)
      elif(data.HA_CLOSE[i] < data.senkou_span_a[i] and data.senkou_span_a[i] > data.senkou_span_b[i]):
          signals.append(-1)
      else:
          signals.append(0)
  data['signals_ich_3'] = signals
  return data
data=signals_ichimoku(data)

## Moving Averages

### Simple Moving Average

In [21]:
def sma(data,period=10):
  data['sma'] = data['HA_CLOSE'].ewm(span=period,adjust=False).mean()
  return data
data=sma(data)

In [ ]:
fig=px.line(data,x="datetime",y=["close","sma"],title="CLOSE VS SMA PRICES")
fig.show()

### Hull Moving Average

In [23]:
def hma(data,period):
 wma_1 = data['HA_CLOSE'].rolling(period//2).apply(lambda x: \
 np.sum(x * np.arange(1, period//2+1)) / np.sum(np.arange(1, period//2+1)), raw=True)
 wma_2 = data['HA_CLOSE'].rolling(period).apply(lambda x: \
 np.sum(x * np.arange(1, period+1)) / np.sum(np.arange(1, period+1)), raw=True)
 diff = 2 * wma_1 - wma_2
 hma = diff.rolling(int(np.sqrt(period))).mean()
 data['hma'] = hma
 return data
data=hma(data,5)

In [ ]:
fig=px.line(data,x="datetime",y=["close","hma"],title="CLOSE VS HMA PRICES")
fig.show()

### signal geneartion using hma and sma

In [25]:
def signals_hma(data):
  signals = []
  for i in range(len(data)):
      if(data['hma'][i] > data['sma'][i]):
          signals.append(1)
      elif(data['hma'][i] < data['sma'][i]):
          signals.append(-1)
      else:
          signals.append(0)
  data['signals_hma'] = signals
  return data
data=signals_hma(data)

## Boilinger Bands

In [26]:
def bollinger_bands(data, BBANDS_window, STDDEV_factor):

    weights = np.arange(1, BBANDS_window + 1)
    weights_sum = np.sum(weights)
    middle_band = np.convolve(data['close'], weights[::-1], mode='valid') / weights_sum
    middle_band = np.pad(middle_band, (len(data) - len(middle_band), 0), 'constant', constant_values=np.nan)
    data['MiddleBand'] = middle_band


    deviations = data['close'] - data['MiddleBand']


    squared_deviations = deviations ** 2
    sum_squared_deviations = np.convolve(squared_deviations, np.ones(BBANDS_window), mode='valid')
    std = np.sqrt(sum_squared_deviations / (BBANDS_window - 1))
    std = np.pad(std, (len(data) - len(std), 0), 'constant', constant_values=np.nan)
    data['Std'] = std


    data['UpperBand'] = data['MiddleBand'] + STDDEV_factor * data['Std']
    data['LowerBand'] = data['MiddleBand'] - STDDEV_factor * data['Std']

    return data
data=bollinger_bands(data,13,2.8)

In [ ]:
fig = go.Figure()

# Add Close price trace
fig.add_trace(go.Scatter(x=data.index, y=data['close'], mode='lines', name='Close',
                         line=dict(color='blue', width=2)))



# Add Upper Bollinger Band trace
fig.add_trace(go.Scatter(x=data.index, y=data['UpperBand'], mode='lines', name='Upper Band',
                         line=dict(color='green', width=2, dash='dash')))

# Add Lower Bollinger Band trace
fig.add_trace(go.Scatter(x=data.index, y=data['LowerBand'], mode='lines', name='Lower Band',
                         line=dict(color='red', width=2, dash='dash')))


# Update layout
fig.update_layout(title='Bollinger Bands',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  template='plotly_dark')

# Show plot
fig.show()

In [28]:
def signals_bollinger(data):
  signals=[0]
  for i in range(1,len(data)):
    if(data.close[i] > data.UpperBand[i] and data.close[i-1] < data.UpperBand[i-1]):
        signals.append(1)
    elif(data.close[i] < data.LowerBand[i] and data.close[i-1] > data.LowerBand[i-1]):
        signals.append(-1)
    else:
        signals.append(0)
  data['signals_bollinger'] = signals
  return data
data=signals_bollinger(data)


## SUPERTREND

In [29]:
# Function to calculate SuperTrend
def calculate_super_trend(data, atr_period, multiplier):
    # Calculate the average of high and low prices
    high_low_avg = (data['HA_HIGH'] + data['HA_LOW']) / 2

    # Calculate the True Range (TR)
    data['tr'] = data['HA_HIGH'] - data['HA_LOW']

    # Calculate the Average True Range (ATR) using rolling mean
    data['atr'] = data['tr'].rolling(atr_period).mean()

    # Calculate the upper and lower bands of the SuperTrend
    data['upper_band'] = high_low_avg + multiplier * data['atr']
    data['lower_band'] = high_low_avg - multiplier * data['atr']

    # Initialize a column to track whether the market is in an uptrend
    data['in_uptrend'] = True

    # Loop through the data to determine the trend direction
    for i in range(1, len(data)):
        # Check if the previous close was above the upper band
        if data['HA_CLOSE'][i - 1] > data['upper_band'][i - 1]:
            data['in_uptrend'][i] = False
        # Check if the previous close was below the lower band
        elif data['HA_CLOSE'][i - 1] < data['lower_band'][i - 1]:
            data['in_uptrend'][i] = True
        else:
            # If neither, maintain the previous trend direction
            data['in_uptrend'][i] = data['in_uptrend'][i - 1]

        # Adjust the bands based on the trend direction
        if data['in_uptrend'][i]:
            data['lower_band'][i] = max(data['lower_band'][i], data['lower_band'][i - 1])
        else:
            data['upper_band'][i] = min(data['upper_band'][i], data['upper_band'][i - 1])

    # Create a 'SuperTrend' column based on the trend direction
    data['SuperTrend'] = np.where(data['in_uptrend'], data['lower_band'], data['upper_band'])

    # Return a DataFrame containing only the 'SuperTrend' column
    return data[['SuperTrend']]

# Calculate SuperTrend using the defined function
data['super'] = calculate_super_trend(data, 12, 4)

# Initialize a 'trend' column in the DataFrame
data['trend'] = 0

# Determine the trend direction based on the relationship between 'close' and 'super'
for i in range(len(data)):
    if data.HA_CLOSE[i] > data.super[i]:
        data.trend[i] = 1  # Set trend to 1 for an uptrend
    elif data.HA_CLOSE[i] < data.super[i]:
        data.trend[i] = -1  # Set trend to -1 for a downtrend

In [ ]:
up_trend = data['close'].where(data['close'] > data['super'])
down_trend = data['close'].where(data['close'] < data['super'])


# Import Plotly for visualization
import plotly.graph_objects as go

# Create a Plotly figure
fig = go.Figure()

# Plot the Close prices
fig.add_trace(go.Scatter(x=data.index, y=data['close'], mode='lines', name='Close', line=dict(color='blue')))

# Plot points above Super Trend in green
fig.add_trace(go.Scatter(x=data.index, y=up_trend, mode='lines', name='Above Super Trend', marker=dict(color='green', size=8)))

# Plot points below Super Trend in red
fig.add_trace(go.Scatter(x=data.index, y=down_trend, mode='lines', name='Below Super Trend', marker=dict(color='red', size=8)))

# Update layout and display the plot
fig.update_layout(
    title='Trend Identification',
    xaxis_title='X-Axis Title',
    yaxis_title='Y-Axis Title',
    showlegend=True,
    template="plotly_dark"
)

## Log Slope

In [31]:
def log_slope(data):
  data['log'] = 0
  for i in range(1,len(data)):
      damn = mt.log(data.close[i]/data.close[i-1])
      data.log[i] = damn
  data['slope'] = data.log.rolling(window = 5).mean()
  data['log_slope'] = (data['slope'][i] - data['slope'].shift(10))/10
  return data
data=log_slope(data)

In [ ]:
fig=px.line(data,x="datetime",y=["log_slope"])
fig.show()

## IBS

In [33]:
def ibs(data):
  data ['IBS'] = (data['HA_CLOSE'] - data['HA_LOW']) /(data['HA_HIGH'] - data['HA_LOW'])
  data['weight_avg'] = data['IBS'].ewm(span=5,adjust=True).mean()
  return data
data=ibs(data)

# Signal Generation

In [34]:
def bs_signals(data):
  signals = []
  for i in range(len(data)):
    ads = data['signals_adx'][i]
    bs = data['signals_bollinger'][i]

    if(bs + ads >= 2 and data.ATR[i] <= 500):
      signals.append(1)

    elif(bs + ads <= -2 and data['ATR'][i] <= 500 ):
      signals.append(-1)

    else:
      signals.append(0)

  data['signals_bs'] = signals
  return data

In [35]:
def ich_signals(data):
    signal = []
    for i in range(len(data)):
      ichs_2 = data['signals_ich_2'][i]
      ichs_3 = data['signals_ich_3'][i]
      hms = data["signals_hma"][i]
      IBS = data['weight_avg'][i]

      if(ichs_3 + ichs_2 + hms>=1 and IBS >= 0.68):
        signal.append(1)

      elif(ichs_3 + ichs_2 + hms <= -1  and IBS <= 0.35):
        signal.append(-1)

      else:
        signal.append(0)

    data['signals_ich'] = signal
    return data

In [36]:
data=bs_signals(data)
data=ich_signals(data)

# RISK MANGEMENT

In [37]:
class RiskMangement():
  def __init__(self,data, signals,stop_loss_percent=0.02,stop_loss_percent_short = 0.02,dynamic_exit_percent=1,multiplier_long = 4, multiplier_short = 10, jump_percent = 5):
    self.data=data.copy()
    self.current=0
    self.signals=signals
    self.signals_final=[]
    self.stop_loss_percent=stop_loss_percent
    self.stop_loss_prcent_short=stop_loss_percent_short
    self.dynamic_exit_percent=dynamic_exit_percent
    self.multiplier_short=multiplier_short
    self.multiplier_long= multiplier_long
    self.jump_percent=jump_percent
    self.jump=0
    self.intial_balance = 1000
    self.capital = self.intial_balance
    self.number_of_stock=0
    self.portfolio_value=[]
    self.stop_loss=0
    self.atr_stop_loss=0
    self.atr_dynamic_exit=0
    self.current_maxima=0
    self.current_minima=0
    self.dynamic_exit=0
    self.multiplier=1
    self.i=0
    self.fall=0
    self.invested_capital=self.capital
    self.entry = 0
    self.exit = 0
    self.current_strategy = 'none'
    self.strategy = 'none'
  def portfolio(self):
    capital=self.capital+self.number_of_stock*self.data["close"][self.i]*self.current
    self.portfolio_value.append(capital)


  def volatility_check(self):
    if(self.data.HA_CLOSE[self.i] < self.data.super[self.i]):
              self.stop_loss_percent_short = 0.02
              self.dynamic_exit_percent = 0.2
              self.multiplier_short = 2
    else:
              self.stop_loss_percent_short = 0.2
              self.dynamic_exit_percent = 1
              self.multiplier_short = 1
  def squareoff(self):
    self.signals_final.append(self.current*-1)
    self.capital=self.capital+self.number_of_stock*self.data["close"][self.i]*self.current
    self.current=0
    self.number_of_stock=0
    self.exit = self.i

  def stop_loss_checker(self):
    if( self.data["close"][self.i]<= self.stop_loss or self.data["close"][self.i] <= self.atr_stop_loss or self.jump>= self.jump_percent):
      return 1
    else: return 0

  def dynamic_exit_checker(self):
    if (self.portfolio_value[-1] >= self.dynamic_exit or
        self.portfolio_value[-1] >= self.atr_dynamic_exit or
        self.jump >= 4 or self.fall >= 1 or self.data.close[self.i] >= self.stop_loss):
        return 1
    else:
        return 0


  def risk_normal(self, strategy):
    if(strategy == 'bollinger'):
      self.signals = self.data['signals_bs']
    elif(strategy == 'ichimoku'):
      self.signals = self.data['signals_ich']
    
    for self.i in range(self.data.shape[0]-1):
          
          if(strategy == 'bollinger' or strategy == 'ichimoku'):
            self.volatility_check()
          self.portfolio()

          if (self.current==1):
            if (self.signals[self.i]==1 or self.signals[self.i]==0):
              self.jump=(self.data["close"][self.i]-self.data["close"][self.i-1])*100/self.data["close"][self.i-1]
              if (self.stop_loss_checker()):
                self.squareoff()
              elif (self.portfolio_value[-1]>self.current_maxima):
                self.signals_final.append(0)
                self.current_maxima=self.portfolio_value[-1]
                self.atr_stop_loss=self.data["close"][self.i]-self.data["ATR"][self.i]*self.multiplier_long
                self.stop_loss=(1-self.stop_loss_percent)*self.data["close"][self.i]
              else :
                self.signals_final.append(0)

            elif (self.signals[self.i]==-1):
              self.squareoff()
        


          elif (self.current==-1):
            if (self.signals[self.i]==-1 or self.signals[self.i]==0):
              self.jump=(self.data["close"][self.i - 1]-self.data["close"][self.i])*100/self.data["close"][self.i-1]
              self.fall=(self.invested_capital-self.portfolio_value[-1])*100/self.invested_capital
              if (self.dynamic_exit_checker()):
                self.squareoff()
              elif(self.portfolio_value[-1]<self.current_minima):
                 self.multiplier=(self.current_minima-self.portfolio_value[-1])/self.current_minima
                 self.current_minima=self.portfolio_value[-1]
                 self.dynamic_exit=(1+self.dynamic_exit_percent)*self.multiplier*self.current_minima
                 self.atr_dynamic_exit=self.portfolio_value[-1]+self.data["ATR"][self.i]*self.multiplier_short
                 self.signals_final.append(0)
              elif(self.portfolio_value[-1]>self.current_minima):
                self.stop_loss=(1+self.stop_loss_percent_short)*self.data["close"][self.i]
                self.signals_final.append(0)
              else: self.signals_final.append(0)
            elif (self.signals[self.i]==1):
                self.squareoff()
          else:
            if(self.signals[self.i]==1):
              self.entry = self.i
              self.current=1
              self.number_of_stock=self.capital/self.data["close"][self.i]
              self.invested_capital=self.capital
              self.stop_loss=(1-self.stop_loss_percent)*self.data['close'][self.i]
              self.atr_stop_loss = self.data.close[self.i] - self.data.ATR[self.i]*self.multiplier_long
              self.current_maxima=self.capital
              self.capital=0
              self.signals_final.append(1)
            elif(self.signals[self.i]==-1):
              self.entry = self.i
              self.current=-1
              self.number_of_stock=self.capital/self.data["close"][self.i]
              self.invested_capital=self.capital
              self.stop_loss=(1+self.stop_loss_percent_short)*self.data['close'][self.i]  #-----------------------------------------
              self.dynamic_exit=(1+self.dynamic_exit_percent)*self.capital

              self.atr_dynamic_exit = self.capital + self.data.ATR[self.i]*self.multiplier_short
              self.current_minima=self.capital
              self.capital=self.capital+self.number_of_stock*self.data.close[self.i]
              self.signals_final.append(-1)
            else:
              self.signals_final.append(0)
    self.portfolio()
    if (self.current!=0):
      self.squareoff()
    else:
      self.signals_final.append(0)
    if(strategy == 'bollinger'):
      self.data["portfolio_value_bs"]=self.portfolio_value
      self.data["final_signals_bs"]=self.signals_final
    elif(strategy == 'ichimoku'):
      self.data["portfolio_value_ich"]=self.portfolio_value
      self.data["final_signals_ich"]=self.signals_final
    






In [38]:
strat1= RiskMangement(data, data['signals_bs'])
strat1.risk_normal('bollinger')
strat2= RiskMangement(data, data['signals_ich'])
strat2.risk_normal('ichimoku')


# Risk Management Advance 
* using alternative mechanism switching between two startegies

In [39]:
class RiskMangementAdvance():
    def __init__(self,data, signal_bs, signal_ich,portfolio_bs,portfolio_ich,stop_loss_percent=0.023,stop_loss_percent_short = 0.02,dynamic_exit_percent=0.2,multi_long = 4, multi_short = 1.2, jump_percent = 5, fall_percent = 1):
        self.data=data.copy()
        self.current=0
        self.signal_bs=signal_bs
        self.signal_ich=signal_ich
        self.signals=self.signal_bs
        self.signals_final=[]
        self.stop_loss_percent=stop_loss_percent
        self.stop_loss_prcent_short=stop_loss_percent_short
        self.dynamic_exit_percent=dynamic_exit_percent
        self.multiplier_short=multi_short
        self.multiplier_long= multi_long
        self.jump_percent=jump_percent
        self.jump=0
        self.intial_balance = 1000
        self.capital = self.intial_balance
        self.number_of_stock=0
        self.portfolio_value=[]
        self.portfolio_value_bs=portfolio_bs
        self.portfolio_value_ich=portfolio_ich
        self.stop_loss=0
        self.atr_stop_loss=0
        self.atr_dynamic_exit=0
        self.current_maxima=0
        self.current_minima=0
        self.dynamic_exit=0
        self.multiplier=1
        self.i=0
        self.fall=0
        self.invested_capital=self.capital
        self.strat=1
    def portfolio(self):
        capital=self.capital+self.number_of_stock*self.data["close"][self.i]*self.current
        self.portfolio_value.append(capital)
    def volatility_check(self):
        if(self.data.HA_CLOSE[self.i] < self.data.super[self.i]):
              self.stop_loss_percent = 0.023
              self.stop_loss_percent_short = 0.2
              self.dynamic_exit_percent = 0.3
              self.multiplier_short = 2
        else:
              self.stop_loss_percent = 0.023
              self.stop_loss_percent_short = 0.2
              self.dynamic_exit_percent = 0.2
              self.multiplier_short = 1.2
    def squareoff(self):
        self.signals_final.append(self.current*-1)
        self.capital=self.capital+self.number_of_stock*self.data["close"][self.i]*self.current
        self.current=0
        self.exit=self.i
        self.number_of_stock=0
        self.switch()   
    def switch(self):
        
        portfolio_bs_return=(self.portfolio_value_bs[self.exit]-self.portfolio_value_bs[self.entry])*100/self.portfolio_value_bs[self.entry]
        portfolio_ich_return=(self.portfolio_value_ich[self.exit]-self.portfolio_value_ich[self.entry])*100/self.portfolio_value_ich[self.entry]
        portfolio_return=(self.capital-self.invested_capital)*100/self.invested_capital
        if self.strat==1:
            if portfolio_ich_return>portfolio_return:
                self.signals=self.signal_ich
                self.strat=2
                
        elif self.strat==2:
            if portfolio_bs_return>portfolio_return:
                self.signals=self.signal_bs
                self.strat=1
                
    def stop_loss_checker(self):
        if( self.data["close"][self.i]<= self.stop_loss or self.data["close"][self.i] <= self.atr_stop_loss or self.jump>= self.jump_percent):
            return 1
        else: return 0
    def dynamic_exit_checker(self):
        if (self.portfolio_value[-1] >= self.dynamic_exit or
            self.portfolio_value[-1] >= self.atr_dynamic_exit or
            self.jump >= 4 or self.fall >= 1 or self.data.close[self.i] >= self.stop_loss):
            return 1
        else:
            return 0
    def risk_normal(self):

        for self.i in range(self.data.shape[0]-1):
            self.volatility_check()
            self.portfolio()
            if (self.current==1):
                if (self.signals[self.i]==1 or self.signals[self.i]==0):
                    self.jump=(self.data["close"][self.i]-self.data["close"][self.i-1])*100/self.data["close"][self.i-1]
                    if (self.stop_loss_checker()):
                        self.squareoff()
                    elif (self.portfolio_value[-1]>self.current_maxima):
                        self.signals_final.append(0)
                        self.current_maxima=self.portfolio_value[-1]
                        self.atr_stop_loss=self.data["close"][self.i]-self.data["ATR"][self.i]*self.multiplier_long
                        self.stop_loss=(1-self.stop_loss_percent)*self.data["close"][self.i]
                    else :
                        self.signals_final.append(0)
                elif (self.signals[self.i]==-1):
                    self.squareoff()
            elif (self.current==-1):
                if (self.signals[self.i]==-1 or self.signals[self.i]==0):
                    self.jump=(self.data["close"][self.i - 1]-self.data["close"][self.i])*100/self.data["close"][self.i-1]
                    self.fall=(self.invested_capital-self.portfolio_value[-1])*100/self.invested_capital
                    if (self.dynamic_exit_checker()):
                        self.squareoff()
                    elif(self.portfolio_value[-1]<self.current_minima):
                        self.multiplier=(self.current_minima-self.portfolio_value[-1])/self.current_minima
                        self.current_minima=self.portfolio_value[-1]
                        self.dynamic_exit=(1+self.dynamic_exit_percent)*self.multiplier*self.current_minima
                        self.atr_dynamic_exit=self.portfolio_value[-1]+self.data["ATR"][self.i]*self.multiplier_short
                        self.signals_final.append(0)
                    elif(self.portfolio_value[-1]>self.current_minima):
                        self.stop_loss=(1+self.stop_loss_percent_short)*self.data["close"][self.i]
                        self.signals_final.append(0)
                    else: self.signals_final.append(0)
                elif (self.signals[self.i]==1):
                    self.squareoff()
            else:
                if(self.signals[self.i]==1):
                    self.entry = self.i
                    self.current=1
                    self.number_of_stock=self.capital/self.data["close"][self.i]
                    self.capital=0
                    self.invested_capital=self.capital
                    self.stop_loss=(1-self.stop_loss_percent)*self.data['close'][self.i]
                    self.atr_stop_loss = self.data.close[self.i] - self.data.ATR[self.i]*self.multiplier_long
                    self.current_maxima=self.capital
                    self.signals_final.append(1)
                elif(self.signals[self.i]==-1):
                    
                    self.entry = self.i
                    self.current=-1
                    self.number_of_stock=self.capital/self.data["close"][self.i]
                    self.invested_capital=self.capital
                    self.stop_loss=(1+self.stop_loss_percent_short)*self.data['close'][self.i]  #--------
                    self.dynamic_exit=(1+self.dynamic_exit_percent)*self.capital
                    self.atr_dynamic_exit = self.capital + self.data.ATR[self.i]*self.multiplier_short
                    self.current_minima=self.capital
                    self.capital=self.capital+self.number_of_stock*self.data.close[self.i]
                    self.signals_final.append(-1)
                else:
                    self.signals_final.append(0)
        self.portfolio()
        if (self.current!=0):
            self.squareoff()
        else:
            self.signals_final.append(0)
        self.data["portfolio_value"]=self.portfolio_value
        self.data["final_signals"]=self.signals_final
    

        

        

In [40]:
strategy= RiskMangementAdvance(data, strat1.data['final_signals_bs'], strat2.data['final_signals_ich'],strat1.data['portfolio_value_bs'],strat2.data['portfolio_value_ich'])
strategy.risk_normal()


# Final OUTPUT CSV with signals

In [41]:
output=pd.DataFrame({'close':strategy.data['close'],'signals':strategy.data['final_signals'],'open':strategy.data['open'],'high':strategy.data['high'],'low':strategy.data['low'],'volume':strategy.data['volume'],'datetime':strategy.data['datetime']})

In [42]:
output.to_csv('output.csv')